# Simple tensor

In [3]:
import numpy as np

class Tensor (object):
    def __init__(self, data):
        self.data = np.array(data)

    def __add__(self, other):
        return Tensor(self.data + other.data)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())
    

x = Tensor([1, 2, 3, 4, 5])
print(x)

y = x + x
print(y)

[1 2 3 4 5]
[ 2  4  6  8 10]


# Tensor with autograd support.

In [6]:
import numpy as np

# Define a class to represent a tensor
class Tensor (object):
    # Initialize the tensor with data, creators, and creation operation
    def __init__(self, data, autograd=False, creators=None, creation_op=None, id=None):
        # Convert the data to a numpy array
        self.data = np.array(data)
        # Store the creation operation (e.g., "add")
        self.creation_op = creation_op
        # Store the creators of this tensor (if any)
        self.creators = creators
        # Initialize the gradient to None
        self.grad = None
        # If autograd is enabled, set the gradient to zero
        self.autograd = autograd
        self.children = {}
        if (id is None):
            id = np.random.randint(0, 100000)
        self.id = id

        if(creators is not None):
            for c in creators:
                # Keeps track of how many children a tensor has
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    # Checks whether a tensor has received the correct number of gradients from each child
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if(self.grad is None):
                return False
            if(cnt > 0):
                return False
        return True
    

    # Perform backpropagation to compute the gradient
    def backward(self, grad=None, grad_origin=None):

        if(self.autograd):
            if(grad_origin is not None):
                # Checks to make sure you can backpropagate or whether you're waiting
                # for a gradient in which case decrement the counter
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                # Accumulates gradients from several children
                self.grad = grad
            else:
                self.grad += grad

            if(self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
                if(self.creation_op == "add"):
                    # Begins actual backpropagation
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)

                if(self.creation_op == "neg"):
                    # Begins actual backpropagation
                    self.creators[0].backward(self.grad.__neg__())

                if(self.creation_op == "sub"):
                    new = Tensor(self.grad.data)
                    self.creators[0].backward(new, self)
                    new = Tensor(self.grad.__neg__().data)
                    self.creators[1].backward(, self)

                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new, self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)

                if(self.creation_op == "mm"):
                    # Usually an aviation
                    act = self.creators[0]
                    # usually a weight matrix
                    weights = self.creators[1]
                    new = self.grad.mm(weights.transpose())
                    weights.backward(new)

                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    ds = self.creators[0].data.shape[dim]
                    self.creators[0].backward(self.grad.expand(dim, ds))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))


    # Overload the "+" operator to add two tensors
    def __add__(self, other):

        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data, autograd=True, creators=[self, other], creation_op="add")

        # Create a new tensor with the sum of the data
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1, autograd=True, creators=[self], creation_op="neg")
        
        return Tensor(self.data * -1)

    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data - other.data, autograd=True, creators=[self, other], creation_op="sub")

        return Tensor(self.data - other.data)

    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data, autograd=True, creators=[self, other], creation_op="mul")
        
        return Tensor(self.data * other.data)

    def sum(self, dim):
        if(self.autograd):
            return Tensor(self.data.sum(dim), autograd=True, creators=[self], creation_op="sum_"+str(dim))
        
        return Tensor(self.data.sum(dim))

    def expand(self, dim, copies):
        trans_cmd = list(range(0, len(self.data.shape)))
        trans_cmd.insert(dim, len(self.data.shape))
        new_shape = list(self.data.shape) + [copies]
        new_data = self.data.repeat(copies).reshape(new_shape)
        new_data = new_data.transpose(trans_cmd)

        if(self.autograd):
            return Tensor(new_data, autograd=True, creators=[self], creation_op="expand_"+str(dim))

        return Tensor(new_data)

    def transpose(self):
        if (self.autograd):
            return Tensor(self.data.transpose(), autograd=True, creators=[self], creation_op="transpose")

        return Tensor(self.data.transpose())

    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data), autograd=True, creators=[self, x], creation_op="mm")

        return Tensor(self.data.dot(x.data))

    # Return a string representation of the tensor
    def __repr__(self):
        return str(self.data.__repr__())

    # Return a string representation of the tensor
    def __str__(self):
        return str(self.data.__str__())

# Create two tensors
x = Tensor([1, 2, 3, 4, 5], autograd=True)
y = Tensor([2, 2, 2, 2, 2], autograd=True)

# Add the two tensors
z = x + y

# Perform backpropagation with a gradient of 1
z.backward(Tensor(np.array([1, 1, 1, 1, 1])))

# Print the gradients of x and y
print(x.grad) 
print(y.grad) 

# Print the creators of z
print(z.creators) 

# Print the creation operation of z
print(z.creation_op)

a = Tensor([1, 2, 3, 4, 5], autograd=True)
b = Tensor([2, 2, 2, 2, 2], autograd=True)
c = Tensor([5, 4, 3, 2, 1], autograd=True)

d = a + (-b)
e = (-b) + c
print(e.creators)
f = d + e
f.backward(Tensor(np.array([1, 1, 1, 1, 1])))
print(b.grad.data == np.array([-2, -2, -2, -2, -2]))

[1 1 1 1 1]
[1 1 1 1 1]
[array([1, 2, 3, 4, 5]), array([2, 2, 2, 2, 2])]
add
[array([-2, -2, -2, -2, -2]), array([5, 4, 3, 2, 1])]
[ True  True  True  True  True]
